# Week 6, Data ingestion  & pipeline 

## Task:

* Take any csv/text file of 2+ GB of your choice. --- (You can do this assignment on Google colab)

* Read the file ( Present approach of reading the file )

* Try different methods of file reading eg: Dask, Modin, Ray, pandas and present your findings in term of computational     efficiency

* Perform basic validation on data columns : eg: remove special character , white spaces from the col name

* As you already know the schema hence create a YAML file and write the column name in YAML file. --define separator of   
  read and write file, column name in YAML

* Validate number of columns and column name of ingested file with YAML.

* Write the file in pipe separated text file (|) in gz format.

* Create a summary of the file:

    Total number of rows,

    total number of columns

    file size
    

## Ingesting big .csv file:
1) Dask

In [48]:
# https://www.geeksforgeeks.org/working-with-large-csv-files-in-python/
#https://www.kaggle.com/code/robikscube/flight-cancellation-dataset-eda

import ray
ray.shutdown()

import numpy as np
import time
import os
from dask import dataframe as df1
  
    
read_path = 'D:/datanew/Combined_Flights_2022.csv'
file_stats = os.stat(read_path)
print("File size being read in GB is {:.2f}".format(file_stats.st_size / (1024 * 1024 * 1024)))

# time taken to read data
s_time_dask = time.time()
dask_df = df1.read_csv(read_path)
e_time_dask = time.time()
  
print("Read with dask: ", round(e_time_dask-s_time_dask,3), "seconds")  
display(dask_df.head(2))

File size being read in GB is 1.32
Read with dask:  0.037 seconds


,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,...,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,...,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0


2) Pandas

In [2]:
import pandas as pd

s_time_pandas = time.time()
pandas_df = pd.read_csv(read_path)
e_time_pandas = time.time()
print("Read with pandas: ", round(e_time_pandas-s_time_pandas,3), "seconds")
#display(pandas_df.head(2))

Read with pandas:  692.9118497371674 seconds


In [7]:
############################## modin ray 
# #https://modin.readthedocs.io/en/stable/getting_started/quickstart.html
import modin.pandas as pdm

ray.init()
s_time_modin = time.time()
modin_df=pdm.read_csv(read_path)
e_time_modin = time.time()
print("Read with modin: ", round(e_time_modin-s_time_modin,3), "seconds")
modin_df.head(10)
#ERROR: MemoryError: Unable to allocate 64.0 KiB for an array with shape (8192,) and data type int64

# https://vaex.io/docs/installing.html
# import vaex
# s_time_vaex = time.time()
# vaex_df = vaex.from_csv(read_path,dtype=dtypetry)
# e_time_vaex = time.time()
# print("Read with vaex: ", (e_time_vaex-s_time_vaex), "seconds")

# # !conda list vaex
# # import vaex as vv
# # #!conda install -c conda-forge vaex


Read with pandas:  740.9041311740875 seconds


,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,...,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,...,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,...,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0
3,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,GPT,False,False,1435,1430.0,0.0,-5.0,...,1446.0,1543.0,4.0,1605,-18.0,0.0,-2.0,1600-1659,2,0
4,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1135,1135.0,0.0,0.0,...,1154.0,1243.0,8.0,1245,6.0,0.0,0.0,1200-1259,2,0
5,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DEN,TUL,False,False,955,952.0,0.0,-3.0,...,1017.0,1234.0,4.0,1240,-2.0,0.0,-1.0,1200-1259,3,0
6,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,LCH,False,False,2139,2136.0,0.0,-3.0,...,2147.0,2213.0,5.0,2231,-13.0,0.0,-1.0,2200-2259,1,0
7,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",TYS,IAH,False,False,1129,1117.0,0.0,-12.0,...,1139.0,1255.0,16.0,1306,5.0,0.0,0.0,1300-1359,4,0
8,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,AEX,False,False,1424,1414.0,0.0,-10.0,...,1430.0,1507.0,6.0,1524,-11.0,0.0,-1.0,1500-1559,1,0
9,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,MOB,False,False,954,947.0,0.0,-7.0,...,1004.0,1104.0,6.0,1121,-11.0,0.0,-1.0,1100-1159,2,0


The column names to add to the YAML file:

In [3]:
pandas_df.columns.values.tolist()


['FlightDate',
 'Airline',
 'Origin',
 'Dest',
 'Cancelled',
 'Diverted',
 'CRSDepTime',
 'DepTime',
 'DepDelayMinutes',
 'DepDelay',
 'ArrTime',
 'ArrDelayMinutes',
 'AirTime',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'Distance',
 'Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'Marketing_Airline_Network',
 'Operated_or_Branded_Code_Share_Partners',
 'DOT_ID_Marketing_Airline',
 'IATA_Code_Marketing_Airline',
 'Flight_Number_Marketing_Airline',
 'Operating_Airline',
 'DOT_ID_Operating_Airline',
 'IATA_Code_Operating_Airline',
 'Tail_Number',
 'Flight_Number_Operating_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRS

## Validation:
    
    - Lower, remove whitespace and replace special characters by _ in the data file read. 
    
    - If the number of columns and their names from the YAML file is the same as that in the data file read, Validation passed.
    
    - Else, point out the different columns in both files 

In [4]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Writing testutility.py


Prepare the YAML file that parmetrize the process of data ingestion

In [5]:
%%writefile ingested_file_info.yaml
file_type: csv
dataset_name: Combined_Flights_2022
file_name: Combined_Flights_2022
table_name: Combined_Flights_2022
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - FlightDate
    - Airline
    - Origin
    - Dest
    - Cancelled
    - Diverted
    - CRSDepTime
    - DepTime
    - DepDelayMinutes
    - DepDelay
    - ArrTime
    - ArrDelayMinutes
    - AirTime
    - CRSElapsedTime
    - ActualElapsedTime
    - Distance
    - Year
    - Quarter
    - Month
    - DayofMonth
    - DayOfWeek
    - Marketing_Airline_Network
    - Operated_or_Branded_Code_Share_Partners
    - DOT_ID_Marketing_Airline
    - IATA_Code_Marketing_Airline
    - Flight_Number_Marketing_Airline
    - Operating_Airline
    - DOT_ID_Operating_Airline
    - IATA_Code_Operating_Airline
    - Tail_Number
    - Flight_Number_Operating_Airline
    - OriginAirportID
    - OriginAirportSeqID
    - OriginCityMarketID
    - OriginCityName
    - OriginState
    - OriginStateFips
    - OriginStateName
    - OriginWac
    - DestAirportID
    - DestAirportSeqID
    - DestCityMarketID
    - DestCityName
    - DestState
    - DestStateFips
    - DestStateName
    - DestWac
    - DepDel15
    - DepartureDelayGroups
    - DepTimeBlk
    - TaxiOut
    - WheelsOff
    - WheelsOn
    - TaxiIn
    - CRSArrTime
    - ArrDelay
    - ArrDel15
    - ArrivalDelayGroups
    - ArrTimeBlk
    - DistanceGroup
    - DivAirportLandings

Writing ingested_file_info.yaml


In [6]:
# Read config file
import testutility as util
config_data = util.read_config_file("ingested_file_info.yaml")
config_data

{'file_type': 'csv',
 'dataset_name': 'Combined_Flights_2022',
 'file_name': 'Combined_Flights_2022',
 'table_name': 'Combined_Flights_2022',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['FlightDate',
  'Airline',
  'Origin',
  'Dest',
  'Cancelled',
  'Diverted',
  'CRSDepTime',
  'DepTime',
  'DepDelayMinutes',
  'DepDelay',
  'ArrTime',
  'ArrDelayMinutes',
  'AirTime',
  'CRSElapsedTime',
  'ActualElapsedTime',
  'Distance',
  'Year',
  'Quarter',
  'Month',
  'DayofMonth',
  'DayOfWeek',
  'Marketing_Airline_Network',
  'Operated_or_Branded_Code_Share_Partners',
  'DOT_ID_Marketing_Airline',
  'IATA_Code_Marketing_Airline',
  'Flight_Number_Marketing_Airline',
  'Operating_Airline',
  'DOT_ID_Operating_Airline',
  'IATA_Code_Operating_Airline',
  'Tail_Number',
  'Flight_Number_Operating_Airline',
  'OriginAirportID',
  'OriginAirportSeqID',
  'OriginCityMarketID',
  'OriginCityName',
  'OriginState',
  'OriginStateFips',
  'OriginSt

In [7]:
# read the file using config file

file_type = config_data['file_type']
source_file ="D:/datanew/" + config_data['file_name'] + f'.{file_type}'
print("",source_file)


df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()



 D:/datanew/Combined_Flights_2022.csv


C:\Users\Rania Fleifel\AppData\Local\Temp\ipykernel_6192\1925505116.py:8: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df = pd.read_csv(source_file,config_data['inbound_delimiter'])


,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,...,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,...,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,...,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,...,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0
3,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",IAH,GPT,False,False,1435,1430.0,0.0,-5.0,...,1446.0,1543.0,4.0,1605,-18.0,0.0,-2.0,1600-1659,2,0
4,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1135,1135.0,0.0,0.0,...,1154.0,1243.0,8.0,1245,6.0,0.0,0.0,1200-1259,2,0


In [8]:
#validate the header of the file
util.col_header_val(df,config_data)

column name and column length validation passed


1

In [10]:
print("columns of files are:" ,df.columns.tolist())
print("columns of YAML are:" ,config_data['columns'])

columns of files are: ['flightdate', 'airline', 'origin', 'dest', 'cancelled', 'diverted', 'crsdeptime', 'deptime', 'depdelayminutes', 'depdelay', 'arrtime', 'arrdelayminutes', 'airtime', 'crselapsedtime', 'actualelapsedtime', 'distance', 'year', 'quarter', 'month', 'dayofmonth', 'dayofweek', 'marketing_airline_network', 'operated_or_branded_code_share_partners', 'dot_id_marketing_airline', 'iata_code_marketing_airline', 'flight_number_marketing_airline', 'operating_airline', 'dot_id_operating_airline', 'iata_code_operating_airline', 'tail_number', 'flight_number_operating_airline', 'originairportid', 'originairportseqid', 'origincitymarketid', 'origincityname', 'originstate', 'originstatefips', 'originstatename', 'originwac', 'destairportid', 'destairportseqid', 'destcitymarketid', 'destcityname', 'deststate', 'deststatefips', 'deststatename', 'destwac', 'depdel15', 'departuredelaygroups', 'deptimeblk', 'taxiout', 'wheelsoff', 'wheelson', 'taxiin', 'crsarrtime', 'arrdelay', 'arrdel15'

In [12]:
if util.col_header_val(df,config_data)==0:
    print("validation failed")
    # write code to reject the file
else:
    print("col validation passed")
    # write the code to perform further action
    # in the pipleine

column name and column length validation passed
col validation passed


## Writing the data file to pipe delimited txt file that is zipped (.gz)

In [11]:
#https://datascience.stackexchange.com/questions/85590/how-do-i-transform-a-file-to-txt-file-using-pandas
df.to_csv('data.txt.gz', sep='|',index=False,compression='gzip')


## Preparing a summary of the data file 

In [45]:
print("The size of the .gz file = {:.2f} MB \nThe size of original csv file = {:.2f} GB ".format(os.stat('data.txt.gz').st_size/(1024*1024),os.stat(source_file).st_size/(1024*1024*1024)))
print("The number of columns = {} \nThe number of rows = {}".format(len(df.columns.tolist()),len(df)))
#df.size=columns*rows

The size of the .gz file = 188.52 MB 
The size of original csv file = 1.32 GB 
The number of columns = 61 
The number of rows = 4078318
